# Example of MRSI fitting on the command line

This notebook demos the process of fitting an MRSI scan using the command line scripts included in FSL-MRS.
### Contents:  
- [1. Reconstruction, Processing and Data Conversion](#1.-Reconstruction,-processing-and-data-conversion)
- [2. Segmentation](#2.-Tissue-segmentation)
- [3. Fitting](#3.-Fitting)
- [4. Visualisation of fit](#4.-Visualisation-of-fit)

Will Clarke  
June 2020  
University of Oxford

## 1. Reconstruction, processing and data conversion

MRSI reconstruction (from k-space data) can be highly specialised depending on the sequence. For example reconstruction of non-cartesian trajectories (e.g. spirals or concentric-rings) requires regridding or use of the NUFFT. Due to the specialised nature of MRSI reconstruction FSL-MRS does not contain tools for this step.

Simmilarly post-processing of MRSI data is commonly done as part of the reconstruction process. Though many of the processing tools in FSL-MRS can be run on MRSI data they have not been created with MRSI in mind.

This example therefore assumes that you are able to provide reconstructed and processed data ready for fitting. Data can be converted to NIfTI from the Siemens DICOM format using spec2nii. The authors of FSL-MRS and spec2nii are happy to add supported formats if example data and interpretation is provided.

## 2. Tissue segmentation

Tissue segmentation is required to have meaningful scaling of water referenced metabolite concentrations. mrsi_segment will produce three files, corresponding to the GM, WM and CSF FAST tissue segmentations registered to the MRSI volume.

Run tissue segmentation on the packaged T1 data and mask using the MRSI data file. Here we provide a (partial) .anat file produced by [fsl_anat](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/fsl_anat) to speed up execution.

This step requires an installation of FSL compatible with fslpy.

In [ ]:
#%%capture
%sx mkdir -p MRSI
%sx mrsi_segment -o MRSI -a example_data/example_mrsi/T1.anat example_data/example_mrsi/mrsi.nii.gz

## 3. Fitting

The below is a call to the main MRSI wrapper script. Note the use of the `--add_MM` flag as this metabolite basis does not contain a macromolecule basis. Also note that the mrsi dataset provided is accompagnied by a JSON sidecar file which contains some useful information such as the echo time, the central frequency, and the dwell time.

The script will by default run in parallel on the available CPU cores. Depending on hardware this should take a few minutes.


In [ ]:
%sx fsl_mrsi --data example_data/example_mrsi/mrsi.nii.gz \
             --basis example_data/example_mrsi/3T_slaser_32vespa_1250.BASIS \
             --output MRSI/example_mrsi_fit \
             --mask example_data/example_mrsi/mask.nii.gz \
             --h2o example_data/example_mrsi/wref.nii.gz \
             --tissue_frac MRSI/mrsi_seg_wm.nii.gz MRSI/mrsi_seg_gm.nii.gz MRSI/mrsi_seg_csf.nii.gz \
             --add_MM \
             --baseline_order 2 \
             --combine PCho GPC --combine Cr PCr --combine NAA NAAG --combine Glu Gln --combine Glc Tau \
             --ignore Gly Gly_1 HG \
             --overwrite

## 4. Visualisation of fit

Now take a look at the outputs. A PNG of the fit to the average of all voxels is provided for a quick sanity check. The folders contain the following:

- concs         : concentration for each metabolite or combined metabolites (subfolders contain different types of referencing)
- fit           : the model prediction FID, the residual FID, and the baseline (also in the time domain).
- qc            : QC parameters split per metabolite
- uncertainties : the std of the fit per metabolite


In [ ]:
%sx ls -l MRSI/example_mrsi_fit

Now run the command below in a terminal in order to load the data in FSLeyes. You will need to install the NIfTI-MRS plugin for FSLeyes. Instructions for installation are available [online](https://open.win.ox.ac.uk/pages/wclarke/fsleyes-plugin-mrs/install.html).

Then follow the the instructions [here](https://open.win.ox.ac.uk/pages/wclarke/fsleyes-plugin-mrs/mrsi_results.html) to set it up in such a way that you can explore each voxel's individual fit.

To get started, after installing the plugin, run the following command:

```
fsleyes -smrs example_data/example_mrsi/T1.anat/T1.nii.gz example_data/example_mrsi/mrsi.nii.gz
```

In [ ]:
%sx fsleyes -smrs example_data/example_mrsi/T1.anat/T1.nii.gz example_data/example_mrsi/mrsi.nii.gz